# CNN Experiments

## 1. Initial Setup

1.1. Import libraries

In [ ]:
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.utils import np_utils
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.callbacks import EarlyStopping
import numpy as np

1.2. Load and preprocess the MNIST data set.

In [ ]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

shape = x_train.shape

# Normalize and reshape the input images
x_train = np.expand_dims(x_train.astype('float32'), -1)
x_test = np.expand_dims(x_test.astype('float32'), -1)

x_train /= 255
x_test /= 255

y_train_class = np_utils.to_categorical(y_train, 10)
y_test_class = np_utils.to_categorical(y_test, 10)

1.3. Create a function to generate a 3-layer CNN, with parameterisable structure, to perform the prediction task.

In [ ]:
INPUT_DIM = x_train.shape[1:]

def create_network(L1_neurons, L2_neurons, **kwargs):
  
  L1_activation = 'relu'
  L2_activation = 'relu'
  L3_neurons = 10
  L3_activation = 'softmax'

  L1_stride = (3,3)
  L2_stride = (3,3)

  L1_pooling = (2,2)
  L2_pooling = (2,2)

  L1_padding = 'same'
  L2_padding = 'same'

  if 'L1_activation' in kwargs.keys():
    L1_activation = kwargs['L1_activation']
  if 'L2_activation' in kwargs.keys():
    L2_activation = kwargs['L2_activation']
  if 'L3_activation' in kwargs.keys():
    L3_activation = kwargs['L3_activation']
  if 'L3_neurons' in kwargs.keys():
    L3_neurons = kwargs['L3_neurons']

  if 'L1_stride' in kwargs.keys():
    L1_stride = kwargs['L1_stride']
  if 'L2_stride' in kwargs.keys():
    L2_stride = kwargs['L2_stride']

  if 'L1_pooling' in kwargs.keys():
    L1_pooling = kwargs['L1_pooling']
  if 'L2_pooling' in kwargs.keys():
    L2_pooling = kwargs['L2_stride']

  model = Sequential()
  model.add(Conv2D(L1_neurons, L1_stride, padding=L1_padding, input_shape=INPUT_DIM))
  model.add(Activation(L1_activation))
  model.add(MaxPooling2D(pool_size=L1_pooling))
  model.add(Conv2D(L2_neurons, L2_stride, padding=L2_padding))
  model.add(Activation(L2_activation))
  model.add(MaxPooling2D(pool_size=L2_pooling))

  model.add(Flatten())
  model.add(Dense(L3_neurons))
  model.add(Activation(L3_activation))

  return model

#Test the function
test_model = create_network(64, 32, L3_neurons = 25)
test_model.summary()

1.4. Write a function which will accept the hyperparameters, generate the neural network and evaluate it against the data set.

In [ ]:
def evaluate(L1_neurons, L2_neurons, **kwargs):
  model = create_network(int(L1_neurons), int(L2_neurons), **kwargs)

  loss_function = 'categorical_crossentropy'
  optimizer = RMSprop(learning_rate=0.0001, decay=1e-6)
  if 'loss_function' in kwargs.keys():
    loss_function = kwargs['loss_function']
  if 'optimizer' in kwargs.keys():
    optimizer = kwargs['optimizer']

  model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])

  batch_size = 32
  if 'batch_size' in kwargs.keys():
    batch_size = int(kwargs['batch_size'])

  history = None
  if 'epochs' in kwargs.keys():
    epochs = int(kwargs['epochs'])
    history = model.fit(x_train, y_train_class, batch_size=batch_size, epochs=epochs, verbose=0)
  else:
    early_stopping = EarlyStopping(monitor='loss', patience=5)
    history = model.fit(x_train, y_train_class, epochs=1000, batch_size=batch_size, callbacks=[early_stopping], verbose=0)

  score = model.evaluate(x_test, y_test_class, verbose=0)

  return score[0]

#Test the function
evaluate(10, 10, L1_pooling=(1,1), L2_pooling=(1,1), L1_stride=(1,1), L2_stride=(1,1), epochs=1)

## 2. Hyperparameter Optimisation Experiments

### 2.A. Setup

#### 2.A.1. Import GitHub Repo

In [ ]:
!git clone https://github.com/aamanrebello/HTVTC-Testing-Framework.git

#Enable importing code from parent directory
import os, sys
final_HTVTC = os.path.abspath('./HTVTC-Testing-Framework/final-HTVTC')
sys.path.insert(1, final_HTVTC)
traditional_methods = os.path.abspath('./HTVTC-Testing-Framework/traditional-methods')
sys.path.insert(1, traditional_methods)
root = os.path.abspath('./HTVTC-Testing-Framework')
sys.path.insert(1, root)

#### 2.A.2. Setup for HTVTC

In [ ]:
!pip install tensorly

#### 2.A.3. Setup for Random Search, BO-TPE, CMA-ES

In [ ]:
!pip install optuna
import optuna

#### 2.A.4. Setup for Hyperband

In [ ]:
!pip install optuna
import optuna

def evaluate_with_budget(L1_neurons, L2_neurons, budget_fraction=1.0, **kwargs):
  model = create_network(int(L1_neurons), int(L2_neurons), **kwargs)

  loss_function = 'categorical_crossentropy'
  optimizer = RMSprop(learning_rate=0.0001, decay=1e-6)
  if 'loss_function' in kwargs.keys():
    loss_function = kwargs['loss_function']
  if 'optimizer' in kwargs.keys():
    optimizer = kwargs['optimizer']

  model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])

  batch_size = 32
  if 'batch_size' in kwargs.keys():
    batch_size = int(kwargs['batch_size'])

  history = None
  training_size = int(budget_fraction*len(x_train))
  x_train_trunc = x_train[:training_size]
  y_train_trunc = y_train_class[:training_size]

  if 'epochs' in kwargs.keys():
    epochs = int(kwargs['epochs'])
    history = model.fit(x_train_trunc, y_train_trunc, batch_size=batch_size, epochs=epochs, verbose=0)
  else:
    early_stopping = EarlyStopping(monitor='loss', patience=5)
    history = model.fit(x_train_trunc, y_train_trunc, epochs=1000, batch_size=batch_size, callbacks=[early_stopping], verbose=0)

  score = model.evaluate(x_test, y_test_class, verbose=0)

  return score[0]

#### 2.A.5. Setup for BOHB-HPO

In [ ]:
!pip install bohb-hpo

def evaluate_with_budget(L1_neurons, L2_neurons, budget_fraction=1.0, **kwargs):
  model = create_network(int(L1_neurons), int(L2_neurons), **kwargs)

  loss_function = 'categorical_crossentropy'
  optimizer = RMSprop(learning_rate=0.0001, decay=1e-6)
  if 'loss_function' in kwargs.keys():
    loss_function = kwargs['loss_function']
  if 'optimizer' in kwargs.keys():
    optimizer = kwargs['optimizer']

  model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])

  batch_size = 32
  if 'batch_size' in kwargs.keys():
    batch_size = int(kwargs['batch_size'])

  history = None
  training_size = int(budget_fraction*len(x_train))
  x_train_trunc = x_train[:training_size]
  y_train_trunc = y_train_class[:training_size]

  if 'epochs' in kwargs.keys():
    epochs = int(kwargs['epochs'])
    history = model.fit(x_train_trunc, y_train_trunc, batch_size=batch_size, epochs=epochs, verbose=0)
  else:
    early_stopping = EarlyStopping(monitor='loss', patience=5)
    history = model.fit(x_train_trunc, y_train_trunc, epochs=1000, batch_size=batch_size, callbacks=[early_stopping], verbose=0)

  score = model.evaluate(x_test, y_test_class, verbose=0)

  return score[0]

#### 2.A.6. Setup for BO-GP

In [ ]:
!pip install git+https://github.com/fmfn/BayesianOptimization

### 2.B. Hyperparameter Optimisation Experiments

#### 2.B.1. HTVTC

In [ ]:
from finalAlgoImplementation import final_HTVTC
import regressionmetrics
import classificationmetrics

metric = classificationmetrics.indicatorFunction

quantity = 'EXEC-TIME'

func = evaluate

def objective(L1_neurons, L2_neurons, L1_activation, L2_activation, L1_pooling_dim, L2_pooling_dim, L1_stride_dim, L2_stride_dim, **kwargs):
  L1_pooling_dim_int = int(L1_pooling_dim)
  L2_pooling_dim_int = int(L2_pooling_dim)
  L1_stride_dim_int = int(L1_stride_dim)
  L2_stride_dim_int = int(L2_stride_dim)
  
  L1_pooling = (L1_pooling_dim_int, L1_pooling_dim_int)
  L2_pooling = (L2_pooling_dim_int, L2_pooling_dim_int)
  L1_stride = (L1_stride_dim_int, L1_stride_dim_int)
  L2_stride = (L2_stride_dim_int, L2_stride_dim_int)

  return func(L1_neurons=L1_neurons, 
                L2_neurons=L2_neurons, 
                L1_activation=L1_activation, 
                L2_activation=L2_activation,
                L1_pooling = L1_pooling,
                L2_pooling = L2_pooling,
                L1_stride = L1_stride,
                L2_stride = L2_stride,
                epochs = kwargs['epochs'])

#Start timer/memory profiler/CPU timer
a = None
start_time = None
if quantity == 'EXEC-TIME':
    import time
    start_time = time.perf_counter_ns()
elif quantity == 'CPU-TIME':
    import time
    start_time = time.process_time_ns()
elif quantity == 'MAX-MEMORY':
    import tracemalloc
    tracemalloc.start()

ranges_dict = {
    'L1_neurons': {
        'type': 'INTEGER',
        'start': 10.0,
        'end': 40.0,
        'interval': 10.0
        },
    'L2_neurons': {
        'type': 'INTEGER',
        'start': 10.0,
        'end': 40.00,
        'interval': 10.0
        },
    'L1_activation': {
        'type': 'CATEGORICAL',
        'values': ['relu', 'tanh', 'sigmoid']
        },
    'L2_activation': {
        'type': 'CATEGORICAL',
        'values': ['relu', 'tanh', 'sigmoid']
        },
    'L1_pooling_dim': {
        'type': 'INTEGER',
        'start': 1.0,
        'end': 5.0,
        'interval': 2.0
        },
    'L2_pooling_dim': {
        'type': 'INTEGER',
        'start': 1.0,
        'end': 5.0,
        'interval': 2.0
        },
    'L1_stride_dim': {
        'type': 'INTEGER',
        'start': 1.0,
        'end': 5.0,
        'interval': 2.0
        },
    'L2_stride_dim': {
        'type': 'INTEGER',
        'start': 1.0,
        'end': 5.0,
        'interval': 2.0
        },
    'epochs': {
        'type': 'CATEGORICAL',
        'values': [1]
    }
  }

recommended_combination, history = final_HTVTC(eval_func=objective, ranges_dict=ranges_dict, metric=metric, max_completion_cycles=4)

#End timer/memory profiler/CPU timer
result = None
if quantity == 'EXEC-TIME':
    end_time = time.perf_counter_ns()
    result = end_time - start_time
elif quantity == 'CPU-TIME':
    end_time = time.process_time_ns()
    result = end_time - start_time
elif quantity == 'MAX-MEMORY':
    _, result = tracemalloc.get_traced_memory()
    tracemalloc.stop()

print('OPT COMPLETED')
#Find the true loss for the selcted combination
true_value1 = func(metric=metric, **recommended_combination)
true_value2 = func(metric=metric, **recommended_combination)
true_value3 = func(metric=metric, **recommended_combination)
true_value4 = func(metric=metric, **recommended_combination)
true_value5 = func(metric=metric, **recommended_combination)

print(f'hyperparameters: {recommended_combination}')
print(f'history: {history}')
print(f'True values: {true_value1}, {true_value2}, {true_value3}, {true_value4}, {true_value5}')
print(f'{quantity}: {result}')

#### 2.B.2. Random Search

In [ ]:
from optuna.samplers import RandomSampler

quantity = 'EXEC-TIME'
func = evaluate

def objective(trial):
    L1_neurons = trial.suggest_int("L1_neurons", 5, 40, step=1)
    L2_neurons = trial.suggest_int("L2_neurons", 5, 40, step=1)
    L1_activation = trial.suggest_categorical("L1_activation", ['relu', 'tanh', 'sigmoid'])
    L2_activation = trial.suggest_categorical("L2_activation", ['relu', 'tanh', 'sigmoid'])
    L1_pooling_dim = trial.suggest_int("L1_pooling_dim", 1, 5, step=1)
    L2_pooling_dim = trial.suggest_int("L2_pooling_dim", 1, 5, step=1)
    L1_stride_dim = trial.suggest_int("L1_stride_dim", 1, 5, step=1)
    L2_stride_dim = trial.suggest_int("L2_stride_dim", 1, 5, step=1)

    L1_pooling = (L1_pooling_dim, L1_pooling_dim)
    L2_pooling = (L2_pooling_dim, L2_pooling_dim)
    L1_stride = (L1_stride_dim, L1_stride_dim)
    L2_stride = (L2_stride_dim, L2_stride_dim)
    
    return func(L1_neurons=L1_neurons, 
                L2_neurons=L2_neurons, 
                L1_activation=L1_activation, 
                L2_activation=L2_activation,
                L1_pooling = L1_pooling,
                L2_pooling = L2_pooling,
                L1_stride = L1_stride,
                L2_stride = L2_stride,
                epochs=1)

#Start timer/memory profiler/CPU timer
start_time = None
if quantity == 'EXEC-TIME':
    import time
    start_time = time.perf_counter_ns()
elif quantity == 'CPU-TIME':
    import time
    start_time = time.process_time_ns()
elif quantity == 'MAX-MEMORY':
    import tracemalloc
    tracemalloc.start()

optuna.logging.set_verbosity(optuna.logging.FATAL)
study = optuna.create_study(sampler=RandomSampler())
study.optimize(objective, n_trials=20)

result = None
if quantity == 'EXEC-TIME':
    end_time = time.perf_counter_ns()
    result = end_time - start_time
elif quantity == 'CPU-TIME':
    end_time = time.process_time_ns()
    result = end_time - start_time
elif quantity == 'MAX-MEMORY':
    _, result = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
print('\n\n\n')
print(f'Number of trials: {len(study.trials)}')
print(f'Best trial: {study.best_trial}')
print(f'{quantity}: {result}')

#### 2.B.3. BO-TPE

In [ ]:
from optuna.samplers import TPESampler

quantity = 'EXEC-TIME'
func = evaluate


def objective(trial):
    L1_neurons = trial.suggest_int("L1_neurons", 5, 40, step=1)
    L2_neurons = trial.suggest_int("L2_neurons", 5, 40, step=1)
    L1_activation = trial.suggest_categorical("L1_activation", ['relu', 'tanh', 'sigmoid'])
    L2_activation = trial.suggest_categorical("L2_activation", ['relu', 'tanh', 'sigmoid'])
    L1_pooling_dim = trial.suggest_int("L1_pooling_dim", 1, 5, step=1)
    L2_pooling_dim = trial.suggest_int("L2_pooling_dim", 1, 5, step=1)
    L1_stride_dim = trial.suggest_int("L1_stride_dim", 1, 5, step=1)
    L2_stride_dim = trial.suggest_int("L2_stride_dim", 1, 5, step=1)

    L1_pooling = (L1_pooling_dim, L1_pooling_dim)
    L2_pooling = (L2_pooling_dim, L2_pooling_dim)
    L1_stride = (L1_stride_dim, L1_stride_dim)
    L2_stride = (L2_stride_dim, L2_stride_dim)
    
    return func(L1_neurons=L1_neurons, 
                L2_neurons=L2_neurons, 
                L1_activation=L1_activation, 
                L2_activation=L2_activation,
                L1_pooling = L1_pooling,
                L2_pooling = L2_pooling,
                L1_stride = L1_stride,
                L2_stride = L2_stride,
                epochs=1)

#Start timer/memory profiler/CPU timer
start_time = None
if quantity == 'EXEC-TIME':
    import time
    start_time = time.perf_counter_ns()
elif quantity == 'CPU-TIME':
    import time
    start_time = time.process_time_ns()
elif quantity == 'MAX-MEMORY':
    import tracemalloc
    tracemalloc.start()

optuna.logging.set_verbosity(optuna.logging.FATAL)
study = optuna.create_study(sampler=TPESampler())
study.optimize(objective, n_trials=20)

#End timer/memory profiler/CPU timer
result = None
if quantity == 'EXEC-TIME':
    end_time = time.perf_counter_ns()
    result = end_time - start_time
elif quantity == 'CPU-TIME':
    end_time = time.process_time_ns()
    result = end_time - start_time
elif quantity == 'MAX-MEMORY':
    _, result = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
print('\n\n\n')
print(f'Number of trials: {len(study.trials)}')
print(f'Best trial: {study.best_trial}')
print(f'{quantity}: {result}')

#### 2.B.4. CMA-ES

In [ ]:
quantity = 'EXEC-TIME'
func = evaluate

def objective(trial):
    L1_neurons = trial.suggest_int("L1_neurons", 5, 40, step=1)
    L2_neurons = trial.suggest_int("L2_neurons", 5, 40, step=1)
    L1_activation = trial.suggest_categorical("L1_activation", ['relu', 'tanh', 'sigmoid'])
    L2_activation = trial.suggest_categorical("L2_activation", ['relu', 'tanh', 'sigmoid'])
    L1_pooling_dim = trial.suggest_int("L1_pooling_dim", 1, 5, step=1)
    L2_pooling_dim = trial.suggest_int("L2_pooling_dim", 1, 5, step=1)
    L1_stride_dim = trial.suggest_int("L1_stride_dim", 1, 5, step=1)
    L2_stride_dim = trial.suggest_int("L2_stride_dim", 1, 5, step=1)

    L1_pooling = (L1_pooling_dim, L1_pooling_dim)
    L2_pooling = (L2_pooling_dim, L2_pooling_dim)
    L1_stride = (L1_stride_dim, L1_stride_dim)
    L2_stride = (L2_stride_dim, L2_stride_dim)
    
    return func(L1_neurons=L1_neurons, 
                L2_neurons=L2_neurons, 
                L1_activation=L1_activation, 
                L2_activation=L2_activation,
                L1_pooling = L1_pooling,
                L2_pooling = L2_pooling,
                L1_stride = L1_stride,
                L2_stride = L2_stride,
                epochs=1)

#Start timer/memory profiler/CPU timer
start_time = None
if quantity == 'EXEC-TIME':
    import time
    start_time = time.perf_counter_ns()
elif quantity == 'CPU-TIME':
    import time
    start_time = time.process_time_ns()
elif quantity == 'MAX-MEMORY':
    import tracemalloc
    tracemalloc.start()

optuna.logging.set_verbosity(optuna.logging.FATAL)
sampler = optuna.samplers.CmaEsSampler()
study = optuna.create_study(sampler=sampler)
study.optimize(objective, n_trials=20)
#resource_usage = getrusage(RUSAGE_SELF)

#End timer/memory profiler/CPU timer
result = None
if quantity == 'EXEC-TIME':
    end_time = time.perf_counter_ns()
    result = end_time - start_time
elif quantity == 'CPU-TIME':
    end_time = time.process_time_ns()
    result = end_time - start_time
elif quantity == 'MAX-MEMORY':
    _, result = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
print('\n\n\n')
print(f'Number of trials: {len(study.trials)}')
print(f'Best trial: {study.best_trial}')
print(f'{quantity}: {result}')

#### 2.B.5. Hyperband

In [ ]:
from commonfunctions import generate_range

quantity = 'EXEC-TIME'
resolution = 0.2

func = evaluate_with_budget

def objective(trial):
    L1_neurons = trial.suggest_int("L1_neurons", 5, 40, step=1)
    L2_neurons = trial.suggest_int("L2_neurons", 5, 40, step=1)
    L1_activation = trial.suggest_categorical("L1_activation", ['relu', 'tanh', 'sigmoid'])
    L2_activation = trial.suggest_categorical("L2_activation", ['relu', 'tanh', 'sigmoid'])
    L1_pooling_dim = trial.suggest_int("L1_pooling_dim", 1, 5, step=1)
    L2_pooling_dim = trial.suggest_int("L2_pooling_dim", 1, 5, step=1)
    L1_stride_dim = trial.suggest_int("L1_stride_dim", 1, 5, step=1)
    L2_stride_dim = trial.suggest_int("L2_stride_dim", 1, 5, step=1)

    L1_pooling = (L1_pooling_dim, L1_pooling_dim)
    L2_pooling = (L2_pooling_dim, L2_pooling_dim)
    L1_stride = (L1_stride_dim, L1_stride_dim)
    L2_stride = (L2_stride_dim, L2_stride_dim)
    metric_value = None

    for fraction in generate_range(resolution,1,resolution):
        metric_value = func(L1_neurons=L1_neurons, 
                L2_neurons=L2_neurons,
                budget_fraction=fraction, 
                L1_activation=L1_activation, 
                L2_activation=L2_activation,
                L1_pooling = L1_pooling,
                L2_pooling = L2_pooling,
                L1_stride = L1_stride,
                L2_stride = L2_stride,
                epochs=1)
        #Check for pruning
        trial.report(metric_value, fraction)
        if trial.should_prune():
            print('=======================================================================================================')
            raise optuna.TrialPruned()

    #Would return the metric for fully trained model (on full dataset)
    return metric_value
    

#Start timer/memory profiler/CPU timer
start_time = None
if quantity == 'EXEC-TIME':
    import time
    start_time = time.perf_counter_ns()
elif quantity == 'CPU-TIME':
    import time
    start_time = time.process_time_ns()
elif quantity == 'MAX-MEMORY':
    import tracemalloc
    tracemalloc.start()

optuna.logging.set_verbosity(optuna.logging.FATAL)
study = optuna.create_study(
    direction="minimize",
    pruner=optuna.pruners.HyperbandPruner(
        min_resource=resolution, max_resource=1, reduction_factor=2
    ),
)
study.optimize(objective, n_trials=5)

result = None
if quantity == 'EXEC-TIME':
    end_time = time.perf_counter_ns()
    result = end_time - start_time
elif quantity == 'CPU-TIME':
    end_time = time.process_time_ns()
    result = end_time - start_time
elif quantity == 'MAX-MEMORY':
    _, result = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
print('\n\n\n')
print(f'Number of trials: {len(study.trials)}')
print(f'Best trial: {study.best_trial}')
print(f'{quantity}: {result}')

#### 2.B.6. BOHB

In [ ]:
from bohb import BOHB
import bohb.configspace as cs

func = evaluate_with_budget
MAXVAL = 10

def objective(fraction, L1_neurons, L2_neurons, L1_activation, L2_activation, L1_pooling_dim, L2_pooling_dim, L1_stride_dim, L2_stride_dim):
    L1_pooling = (L1_pooling_dim, L1_pooling_dim)
    L2_pooling = (L2_pooling_dim, L2_pooling_dim)
    L1_stride = (L1_stride_dim, L1_stride_dim)
    L2_stride = (L2_stride_dim, L2_stride_dim)
    return func(L1_neurons=L1_neurons, 
                L2_neurons=L2_neurons,
                budget_fraction=fraction, 
                L1_activation=L1_activation, 
                L2_activation=L2_activation,
                L1_pooling = L1_pooling,
                L2_pooling = L2_pooling,
                L1_stride = L1_stride,
                L2_stride = L2_stride,
                epochs=1)
    
def evaluate(params, n_iterations):
    fraction = n_iterations/MAXVAL
    return objective(**params, fraction=fraction)


if __name__ == '__main__':
    L1_neurons = cs.IntegerUniformHyperparameter('L1_neurons', 5, 40)
    L2_neurons = cs.IntegerUniformHyperparameter('L2_neurons', 5, 40)
    L1_activation = cs.CategoricalHyperparameter('L1_activation', ['relu', 'tanh', 'sigmoid'])
    L2_activation = cs.CategoricalHyperparameter('L2_activation', ['relu', 'tanh', 'sigmoid'])
    L1_pooling_dim = cs.IntegerUniformHyperparameter('L1_pooling_dim', 1, 5)
    L2_pooling_dim = cs.IntegerUniformHyperparameter('L2_pooling_dim', 1, 5)
    L1_stride_dim = cs.IntegerUniformHyperparameter('L1_stride_dim', 1, 5)
    L2_stride_dim = cs.IntegerUniformHyperparameter('L2_stride_dim', 1, 5)
    configspace = cs.ConfigurationSpace([L1_neurons, L2_neurons, L1_activation, L2_activation, L1_pooling_dim, L2_pooling_dim, L1_stride_dim, L2_stride_dim])

    opt = BOHB(configspace, evaluate, max_budget=MAXVAL, min_budget=1, eta=2)

    quantity = 'EXEC-TIME'

    #Start timer/memory profiler/CPU timer
    start_time = None
    if quantity == 'EXEC-TIME':
        import time
        start_time = time.perf_counter_ns()
    elif quantity == 'CPU-TIME':
        import time
        start_time = time.process_time_ns()
    elif quantity == 'MAX-MEMORY':
        import tracemalloc
        tracemalloc.start()

    logs = opt.optimize()

    #End timer/memory profiler/CPU timer
    result = None
    if quantity == 'EXEC-TIME':
        end_time = time.perf_counter_ns()
        result = end_time - start_time
    elif quantity == 'CPU-TIME':
        end_time = time.process_time_ns()
        result = end_time - start_time
    elif quantity == 'MAX-MEMORY':
        _, result = tracemalloc.get_traced_memory()
        tracemalloc.stop()
    
    print(logs)
    print(f'{quantity}: {result}')

#### 2.B.7. BO-GP

In [ ]:
from bayes_opt import BayesianOptimization

quantity = 'EXEC-TIME'
trials = 15
pval = 1

func = evaluate

def classify_activation(value):
   if value < 5:
      return 'relu'
   elif value < 10:
      return 'tanh'
   else:
      return 'sigmoid'

def objective(L1_neurons, L2_neurons, L1_activation, L2_activation, L1_pooling_dim, L2_pooling_dim, L1_stride_dim, L2_stride_dim):
    L1_neu_int = int(L1_neurons)
    L2_neu_int = int(L2_neurons)

    L1_act_str = classify_activation(L1_activation)
    L2_act_str = classify_activation(L2_activation)

    L1_pooling_dim_int = int(L1_pooling_dim)
    L2_pooling_dim_int = int(L2_pooling_dim)
    L1_stride_dim_int = int(L1_stride_dim)
    L2_stride_dim_int = int(L2_stride_dim)

    L1_pooling = (L1_pooling_dim_int, L1_pooling_dim_int)
    L2_pooling = (L2_pooling_dim_int, L2_pooling_dim_int)
    L1_stride = (L1_stride_dim_int, L1_stride_dim_int)
    L2_stride = (L2_stride_dim_int, L2_stride_dim_int)

    #subtract from 1 because the library only supports maximise
    funcval = func(L1_neurons=L1_neurons, 
                L2_neurons=L2_neurons, 
                L1_activation=L1_act_str, 
                L2_activation=L2_act_str,
                L1_pooling = L1_pooling,
                L2_pooling = L2_pooling,
                L1_stride = L1_stride,
                L2_stride = L2_stride,
                epochs=1)
    return pval - funcval

#Start timer/memory profiler/CPU timer
start_time = None
if quantity == 'EXEC-TIME':
    import time
    start_time = time.perf_counter_ns()
elif quantity == 'CPU-TIME':
    import time
    start_time = time.process_time_ns()
elif quantity == 'MAX-MEMORY':
    import tracemalloc
    tracemalloc.start()

#Begin optimisation
pbounds = {
    'L1_neurons': (5, 40), 
    'L2_neurons': (5, 40), 
    'L1_activation': (0, 15), 
    'L2_activation': (0, 15),
    'L1_pooling_dim': (1,5),
    'L2_pooling_dim': (1,5),
    'L1_stride_dim': (1,5),
    'L2_stride_dim': (1,5),
    }

optimizer = BayesianOptimization(
    f=objective,
    pbounds=pbounds,
    random_state=1,
    verbose = 0
)

optimizer.maximize(
    init_points=10,
    n_iter=trials,
)

result = None
if quantity == 'EXEC-TIME':
    end_time = time.perf_counter_ns()
    result = end_time - start_time
elif quantity == 'CPU-TIME':
    end_time = time.process_time_ns()
    result = end_time - start_time
elif quantity == 'MAX-MEMORY':
    _, result = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
best = optimizer.max
best_params = best['params']
best_score = pval - best['target']
print(f'Number of trials: {trials}')
print(f'Best params: {best_params}')
print(f'Best score: {best_score}')
print(f'{quantity}: {result}')

### 2.C. Display Background Specifications

In [ ]:
!lscpu

In [ ]:
!nvidia-smi -L